In [97]:
import pandas as pd
import plotly.express as px

In [98]:
# https://health.data.ny.gov/Health/Health-Facility-Certification-Information/2g9y-7kqm
# https://health.data.ny.gov/Health/New-York-State-Population-Data-Beginning-2003/e9uj-s3sf

ny_file = open("data/New_York_State_Population_Data__Beginning_2003.csv",'r')
ny_file_out = open("data/New_York_State_Population_Data__Beginning_2003_clean.csv",'w')
for l in ny_file:
    if "Total" not in l:
        ny_file_out.write(l)
ny_file.close()
ny_file_out.close()

In [99]:
ny_pop = pd.read_csv("data/New_York_State_Population_Data__Beginning_2003_clean.csv")
hf_info = pd.read_csv("data/Health_Facility_Certification_Information.csv")

In [100]:
hf_info["Effective Date"]=hf_info["Effective Date"].astype('datetime64[ns]')
d='2018-01-01'
date=pd.to_datetime(d)
hf_info = hf_info[hf_info["Effective Date"] < date] # only consider stuff before 2017
hf_info = hf_info[hf_info["Sub Type"] == "Permanent"] # consider only permanent situation
hf_info.drop(["Regional Office", "Effective Date"], axis=1,inplace=True)

In [101]:
# print(hf_info.columns)
bed_county_attval = hf_info.groupby(by=['Attribute Value', 'County'])["Measure Value"].sum().reset_index()


In [102]:
#avg population over all the years
ny_pop.loc[ny_pop["Race/Ethnicity Description"] != "White Non Hispanic", "Race/Ethnicity Description" ] = "POC"
ny_pop = ny_pop[(ny_pop["County Name"] != "New York State") & (ny_pop["County Name"] != "Rest of State") ]
ny_pop_grouped = ny_pop.groupby(by=["Year", "County Name","Race/Ethnicity Description"])["Population"].sum().reset_index().groupby(by=["County Name","Race/Ethnicity Description"])["Population"].mean()
ny_pop_grouped_reindex =    ny_pop_grouped.reset_index().sort_values(by=["Population"])
fig = px.bar(ny_pop_grouped_reindex, x="County Name", y="Population", color="Race/Ethnicity Description",
             barmode = 'stack')

fig.show()


In [103]:
total_nyc_pop = ny_pop_grouped_reindex.Population.sum()
total_poc_pop = ny_pop_grouped_reindex[ny_pop_grouped_reindex["Race/Ethnicity Description"] == "POC"].Population.sum()
total_white_pop = total_nyc_pop - total_poc_pop
# print(total_nyc_pop,total_poc_pop/total_nyc_pop, total_white_pop/total_nyc_pop)
county_dist_perc = ny_pop_grouped / ny_pop_grouped_reindex.groupby(by="County Name").Population.sum()
county_dist_perc_indxed = county_dist_perc.reset_index()
std_county_pop_perc = county_dist_perc_indxed.Population.std()

#Asset 1

This visualization lists the different counties of New York State and their population attributes. A point to focus on is that counties are separated into People of Color (POC) and White Non-Hispanic (WNH).
The POC category is based on counties having a population of POC one standard deviations above the mean POC population of New York State. WNH counties are categorized as counties having a WNH population 0.5 standard deviations above the mean WNH population of New York State.
I took one standard deviations as my bar as I wanted to look at counties with robust differences in racial/ethnic makeup. One sees that there are 31 WNH counties as compared to just 1 POC counties while the total population is very similar.

In [104]:
top_counties = county_dist_perc_indxed[((county_dist_perc_indxed["Race/Ethnicity Description"] == "POC")  &
                        (county_dist_perc_indxed["Population"] > (total_poc_pop/total_nyc_pop + std_county_pop_perc))) |
                        ((county_dist_perc_indxed["Race/Ethnicity Description"] != "POC")  &
                        (county_dist_perc_indxed["Population"] > (total_white_pop/total_nyc_pop + std_county_pop_perc)))]

top_counties = pd.merge(top_counties,ny_pop_grouped_reindex,on=["County Name","Race/Ethnicity Description"], how="inner",suffixes=["_perc",""])
top_counties_trimmed = top_counties#[top_counties["Population"] > 3e+05]
print(top_counties_trimmed)
print(top_counties_trimmed.groupby(by="Race/Ethnicity Description").agg({'Population': ['count', 'sum']}))


    County Name Race/Ethnicity Description  Population_perc    Population
0      Allegany         White Non Hispanic         0.958987  4.691413e+04
1         Bronx                        POC         0.883273  1.238669e+06
2   Cattaraugus         White Non Hispanic         0.929084  7.433160e+04
3        Cayuga         White Non Hispanic         0.921665  7.355640e+04
4    Chautauqua         White Non Hispanic         0.903530  1.207832e+05
5      Chenango         White Non Hispanic         0.962268  4.840400e+04
6       Clinton         White Non Hispanic         0.918183  7.501480e+04
7      Cortland         White Non Hispanic         0.948202  4.614533e+04
8      Delaware         White Non Hispanic         0.937894  4.374620e+04
9         Essex         White Non Hispanic         0.933134  3.597780e+04
10       Fulton         White Non Hispanic         0.943871  5.178353e+04
11      Genesee         White Non Hispanic         0.927403  5.475680e+04
12     Hamilton         White Non Hisp

The same analysis as above but with 0.5 standard deviation above the mean. We see the same trend that the number counties categorized as WNH is greater than POC.

In [105]:
top_counties = county_dist_perc_indxed[((county_dist_perc_indxed["Race/Ethnicity Description"] == "POC")  &
                        (county_dist_perc_indxed["Population"] > (total_poc_pop/total_nyc_pop + 0.5*std_county_pop_perc))) |
                        ((county_dist_perc_indxed["Race/Ethnicity Description"] != "POC")  &
                        (county_dist_perc_indxed["Population"] > (total_white_pop/total_nyc_pop + 0.5*std_county_pop_perc)))]

top_counties = pd.merge(top_counties,ny_pop_grouped_reindex,on=["County Name","Race/Ethnicity Description"], how="inner",suffixes=["_perc",""])
top_counties_trimmed = top_counties#[top_counties["Population"] > 3e+05]
print(top_counties_trimmed)
print(top_counties_trimmed.groupby(by="Race/Ethnicity Description").agg({'Population': ['count', 'sum']}))

    County Name Race/Ethnicity Description  Population_perc    Population
0        Albany         White Non Hispanic         0.773165  2.341735e+05
1      Allegany         White Non Hispanic         0.958987  4.691413e+04
2         Bronx                        POC         0.883273  1.238669e+06
3        Broome         White Non Hispanic         0.876559  1.726240e+05
4   Cattaraugus         White Non Hispanic         0.929084  7.433160e+04
5        Cayuga         White Non Hispanic         0.921665  7.355640e+04
6    Chautauqua         White Non Hispanic         0.903530  1.207832e+05
7       Chemung         White Non Hispanic         0.889215  7.851860e+04
8      Chenango         White Non Hispanic         0.962268  4.840400e+04
9       Clinton         White Non Hispanic         0.918183  7.501480e+04
10     Columbia         White Non Hispanic         0.892261  5.563233e+04
11     Cortland         White Non Hispanic         0.948202  4.614533e+04
12     Delaware         White Non Hisp

#Asset 2

This visualization looks at maternity beds/services available per 10000 people in counties categorized as POC and counties categorized as WNH.
POC counties have an average of 1.4 maternity beds/services per 10000 people whereas WNH counties have 2 maternity beds/services per 10000 people. Both these numbers are low, but the visualization concludes that POC categorized counties have lesser number of maternity beds/services.

In [106]:
bed_county_attval_copy = bed_county_attval[bed_county_attval["County"].isin(top_counties["County Name"])]
# bed_county_attval_copy.loc[bed_county_attval_copy[]]
all_merged = pd.merge(top_counties_trimmed,bed_county_attval_copy,how="inner",left_on=["County Name"],right_on=["County"])
# all_merged_trimmed = all_merged[all_merged["Population"] > 2e+05]

all_merged = all_merged[all_merged["Attribute Value"] == "Maternity"]
all_merged["att_value_pop"] = all_merged["Measure Value"] *10000/ all_merged["Population"]
# print(all_merged)
all_merged.groupby(by=["Race/Ethnicity Description", "County"])["att_value_pop"].sum()
temp = all_merged.groupby(by=["Race/Ethnicity Description", "County"])["att_value_pop"].sum().reset_index().sort_values(by="att_value_pop")
fig = px.bar(temp, x="County", y="att_value_pop", color="Race/Ethnicity Description",
             barmode = 'stack')
avg_att_val = temp.groupby(by=["Race/Ethnicity Description"])["att_value_pop"].mean()
# print(avg_att_val)
# fig.add_hline(y=temp[temp["County"] == "Queens"]["att_value_pop"].values[0],line_color="red")
fig.add_hline(y=avg_att_val.values[0],line_color="red")
fig.add_hline(y=avg_att_val.values[1],line_color="blue")
fig.update_yaxes(title="Maternity beds/services")
fig.show()


